In [3]:

import os

import cv2

from numpy import *

from keras.models import Sequential

from keras.layers import Input,Dense

from keras.layers import Conv2D, MaxPooling2D, Flatten

from keras.layers import Conv2D, SeparableConv2D, MaxPooling2D, GlobalAveragePooling2D, GlobalMaxPooling2D,BatchNormalization

from keras.optimizers import SGD

from keras.utils import np_utils

from keras.utils.vis_utils import plot_model

import numpy as np

import matplotlib
import matplotlib.pyplot as plt
import pydot
from keras.layers import Convolution2D, MaxPooling2D,Concatenate
from keras.models import Model

from sklearn.preprocessing import LabelEncoder
from skimage import io, transform
from sklearn.metrics import accuracy_score
from scipy import misc

c=np.load('icar(l).npz')
train_data=c['train_data']
train_labels=c['train_labels']
test_data=c['test_data']
test_labels=c['test_labels']


train_data=train_data.astype(np.float16)/255
test_data=test_data.astype(np.float16)/255
#将标签量进行转化
train_labels=np_utils.to_categorical(train_labels)
test_labels=np_utils.to_categorical(test_labels)

def net2(nb_classes):
#model = Sequential()
    img_input = Input(shape=(50,50,1))

    conv1=Conv2D(filters=16, kernel_size=(5,5), padding='same', activation='relu')(img_input)
    
    conv1= BatchNormalization()(conv1)

    maxpool1=MaxPooling2D(pool_size=(2,2))(conv1)

    conv2=Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu')(maxpool1)
    
    conv2= BatchNormalization()(conv2)

    maxpool2=MaxPooling2D(pool_size=(2,2))(conv2)

    conv3=Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu')(maxpool2)
    
    conv3= BatchNormalization()(conv3)

    maxpool3=MaxPooling2D(pool_size=(2,2))(conv3)

    conv4=Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu')(maxpool3)
    
    conv4= BatchNormalization()(conv4)

    maxpool4=MaxPooling2D(pool_size=(2,2))(conv4)

    fire_squeeze = Convolution2D(
        16, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire2_squeeze')(maxpool4)
    
    fire_expand1 = Convolution2D(
        64, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire2_expand1')(fire_squeeze)
    
    fire_expand2 = Convolution2D(
        64, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire2_expand2')(fire_squeeze)
    
    merge = Concatenate(axis=1)([fire_expand1, fire_expand2])
    
    x = GlobalAveragePooling2D()(merge)
    
    x = Dense(nb_classes, activation='softmax')(x)
   
    model = Model(img_input, x, name='net2')
    return model

    #GlobalAveragePooling2D()(x)
    # model.add(Dense(120, activation='relu'))

    # model.add(Dense(84, activation='relu'))

    #model.add(Dense(4, activation='softmax'))
nb_classes=4
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model=net2(nb_classes)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

history=model.fit(train_data, train_labels,validation_split=0.2, batch_size=32, epochs=20, verbose=1, shuffle=True)

#测试模型
preds = np.argmax(model.predict(test_data), axis=1)
test_labels = np.argmax(test_labels, axis=1)
print (accuracy_score(test_labels, preds))

#记录模型日志
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))
plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=False)
#画模型准确率曲线和损失率曲线
plt.plot(epochs, acc, 'ro:', label='Training acc')
plt.plot(epochs, val_acc, 'bo:', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'ro:', label='Training loss')
plt.plot(epochs, val_loss, 'bo:', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()
plt.savefig('graph.png')

ValueError: Error when checking input: expected input_1 to have 4 dimensions, but got array with shape (1600, 50, 50)